In [1]:
from Grid_to_Graph.GRDECL_Parser import GRDECL_Parser
from Grid_to_Graph.struct_discretizer import StructDiscretizer
import numpy as np
import graph_coarsening.graph_utils as graph_utils
#import graph_coarsening.useful_custom_functions as useful_custom_functions
from graph_coarsening.coarsening_utils import *
from collections import defaultdict
from operator import itemgetter
import pandas as pd
import fileinput
import sys
import os
from pygsp import graphs
%matplotlib inline
# %matplotlib notebook
# %pylab
plt.rcParams['figure.figsize'] = (11, 9)

In [2]:
def OneDim_to_ThreeDim_Converter(i_max, j_max, k_max, One_Dim):
    # приводим массив к 3D размерности грида.
    Three_Dim = np.array(np.zeros((i_max, j_max, k_max)))
    i = 0
    j = 0
    k = 0
    z = 0

    while k <= k_max - 1:
        while j <= j_max - 1:
            while i <= i_max - 1:
                Three_Dim[i, j, k] = One_Dim[z]
                i += 1
                z += 1
            i = 0
            j += 1
        j = 0
        k += 1

    return Three_Dim


def get_coarsening(Gall, Call, size=5, node_size=20, title=""):
    edges_s = {}

    n_levels = len(Gall) - 1
    for level in range(n_levels):
        G = Gall[level]
        edges = np.array(G.get_edge_list()[0:2])

        Gc = Gall[level + 1]
        edges_c = np.array(Gc.get_edge_list()[0:2])
        C = Call[level]
        C = C.toarray()

        for i in range(Gc.N):
            subgraph = np.arange(G.N)[C[i, :] > 0]
            edges_s[i] = subgraph
            #if np.clip(len(subgraph) - 1, 0, 4) > 0:
            #edges_s[i] = subgraph

    return edges_s


def replaceAll(file, searchExp, replaceExp):    
    for line in fileinput.input(file, inplace=1):
        if searchExp in line:
            line = line.replace(searchExp,replaceExp)
        sys.stdout.write(line)

Считываем данные по гриду

In [3]:
# from grid2graph.data import *
# dataset_list = get_dataset('dataset/', tensor_path='dataset/tensor.pt', i_max=60, j_max=60, z_max=1)
# dataset_list

In [4]:
# poro = []
# perm = []

# for d in tqdm(dataset_list):
#     poro.append(d.x[:, 0].cpu())
#     perm.append(d.x[:, 1].cpu())

# poro = np.concatenate((poro), axis=0)
# permx = np.concatenate((perm), axis=0)

In [5]:
import pandas as pd

df = pd.read_csv('dataset/EGG.csv')

In [6]:
coords = np.array(OneDim_to_ThreeDim_Converter(60, 60, 1, np.arange(3600) - 1))
coords_new = np.zeros((3600, 3))
for i in range(60):
    for j in range(60):
        for k in range(1):
            if coords[i, j, k] != -1:
                coords_new[int(coords[i, j, k])] = np.array([i, j, k])

In [7]:
dx = 8 
dy = 8
dz = 4



actnum = df['ACTNUM'].to_numpy() #dataset_list[0].label.cpu().numpy()
nx = 60
ny = 60
nz = 1

coord = coords_new

zcorn = 0

permx = df['PERM'].to_numpy() #grid_data.SpatialDatas['PERMX'] * 100
permx[permx<=0] = 0.0002
permy = permx
permz = permy

poro = df['PORO'].to_numpy()  #grid_data.SpatialDatas['PORO']
poro[poro<=0] = 0.0002

volume = np.zeros_like(poro)
volume.fill(dx*dy*dz)

depth = np.zeros_like(poro)
depth.fill(1000)

n = nx * ny * nz

In [8]:
# df = pd.DataFrame([])
# df['PORO'] = poro
# df['PERM'] = permx
# df['ACTNUM'] = actnum

# df.to_csv('dataset/EGG.csv')


In [9]:

# # Визуализируем первый грид на первом subplot
# def vis(grid1):
#     fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(6, 6))

#     # Визуализируем первый грид на первом subplot
#     x, y, z = np.meshgrid(np.arange(grid1.shape[0]),
#                           np.arange(grid1.shape[1]),
#                           np.arange(grid1.shape[2]))
#     print(x.ravel())
    
#     ax1.scatter(x.ravel(), y.ravel(), z.ravel(), c=grid1.flatten(), vmin=0, vmax=1)

#     plt.show()


# vis(discretizer.convert_to_3d_array(poro, 'PORO'))

In [10]:
# discretizer.convert_to_3d_array(poro, 'PORO').ravel()

Получаем conection list

In [11]:
discretizer = StructDiscretizer(nx=nx, ny=ny, nz=nz, dx=dx, dy=dy, dz=dz, permx=permx, permy=permy,
                                             permz=permz, coord=coord, zcorn=zcorn)

if discretizer.is_cpg:
    cell_m, cell_p, tran, tran_thermal = discretizer.calc_cpg_discr()
else:
    cell_m, cell_p, tran, tran_thermal = discretizer.calc_structured_discr()

Building connection list...


In [12]:
permx.shape , poro.shape , depth.shape, volume.shape

((3600,), (3600,), (3600,), (3600,))

In [13]:
# apply actnum filter if needed - all arrays providing a value for a single grid block should be passed
op_num = 0
arrs = [poro, depth, volume, op_num]

cell_m, cell_p, tran, tran_thermal, arrs_local = discretizer.apply_actnum_filter(actnum, cell_m,
                                                                                 cell_p, tran,
                                                                                 tran_thermal, arrs)
poro, depth, volume, op_num = arrs_local

permx = permx[actnum != 0]

Applying ACTNUM...
Inactive blocks due to ACTNUM:  1109
Inactive connections due to transmissibility:  0
Inactive connections total:  2213
Inactive blocks due to inactive connections:  0


Создаём граф

In [14]:
# n = nx*ny*nz
# graph_W = np.zeros((n, n))
# for i, j, t in zip(cell_m, cell_p, tran):
#     if t > 0:
#         w = 1/t
#     else:
#         w = 1/0.0001
#     graph_W[i][j] = w
#     graph_W[j][i] = w

In [15]:
edges = list(zip(cell_m, cell_p))
#weights = [1/t if t>0 else 1/0.0001 for t in tran]
weights = [t for t in tran]

# Количество вершин в графе
num_vertices = len(set(sum(edges, ())))

# Создание матрицы смежности
adjacency_matrix = np.zeros((num_vertices, num_vertices))
for edge, weight in zip(edges, weights):
    adjacency_matrix[edge[0], edge[1]] = weight
    adjacency_matrix[edge[1], edge[0]] = weight

In [16]:
coords = np.array(OneDim_to_ThreeDim_Converter(nx, ny, nz, np.arange(n) - 1))
coords_new = np.zeros((n, 3))
for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            if coords[i, j, k] != -1:
                coords_new[int(coords[i, j, k])] = np.array([i, j, k])

coords_new = coords_new[actnum != 0]

In [17]:
N = coords_new.shape[0]

G = graphs.Graph(adjacency_matrix)

G.set_coordinates(coords_new)


N = G.N
L = G.L.toarray()
S = graph_utils.get_S(G).T

Загрубляем граф

In [19]:
method = 'heavy_edge'

# Parameters
r = 0.25   # the extend of dimensionality reduction (r=0 means no reduction)
k = 10
kmax = int(3 * k)

C, Gc, Call, Gall, coarsening_list, iC, err_list, r_list = coarsen(G, K=k, r=r, method=method, 
                            #algorithm = "optimal"
                            )
#metrics = coarsening_quality(G, C, kmax=kmax)
n = Gc.N

# print('{:16} | r: {:1.4}, nedges: {}, levels: {}, epsilon: {:1.4}'.format(method, metrics['r'], metrics['m'], len(Call),
#                                                                           metrics['error_subspace'][k - 1]))




0.00026722915727242166
0.000554331480478245
0.0006144508361790685
0.0008092547593199238
0.0011063643191778928
0.0012854185273420463
0.0013169531422097826
0.001428064816595492
0.0016277906401528282
0.002163420094201172
0.0023676131214879757
0.0026179560048843692
0.003055306293962799
0.0032436413138219613
0.003433730080513145
0.003626259943547867
0.004268480377964571
0.004819939639235519
0.0049964149972199215
0.005575069419869531
0.005631948133683949
0.005905433398936861
0.005997980581722294
0.006283851645204876
0.006539702818720241
0.006772879644324073
0.006988386187232183
0.007048233268192915
0.007655151883245141
0.007726674619927402
0.008940422970021306
0.009188914156080901
0.009458119321048019
0.009648111803943801
0.009884305804299963
0.010184686127975495
0.010601624672568115
0.010809603034970383
0.011591761904189831
0.012413114932648952
0.012553036047289644
0.012682524635465852
0.013017443665779078
0.013158797217999521
0.014444397051869581
0.014796221635359758
0.014836179553157053
0

In [ ]:
plt.plot(r_list, err_list, '-')


In [ ]:
metrics = coarsening_quality(G, C, kmax=k)
np.nanmean(metrics['error_eigenvalue']) 

In [ ]:
Gall[1].coords[:,[0]].shape

In [ ]:
plt.scatter(coords_new[:,[0]], coords_new[:,[1]])

In [ ]:
plt.scatter(Gall[1].coords[:,[0]], Gall[1].coords[:,[1]])

In [ ]:
def coarsening_quality_(G, C, kmax=30, Uk=None, lk=None):
    if (Uk is not None) and (lk is not None) and (len(lk) >= kmax):
        U, l = Uk, lk
    elif hasattr(G, "U"):
        U, l = G.U, G.e
    else:
        l, U = sp.sparse.linalg.eigsh(G.L, k=kmax, which="SM", tol=1e-3)

    l[0] = 1
    linv = l ** (-0.5)
    linv[0] = 0
    n = C.shape[0]
    Lc = C.dot((G.L).dot(C.T))

    # if kmax > n / 2:
    #     [Uc, lc] = graph_utils.eig(Lc.toarray())
    # else:
    lc, Uc = sp.sparse.linalg.eigsh(Lc, k=kmax, which="SM", tol=1e-3)

    if not sp.sparse.issparse(Lc):
        print("warning: Lc should be sparse.")

    metrics = {}
    kmax = np.clip(kmax, 1, n)

    # eigenvalue relative error
    metrics["error_eigenvalue"] = np.abs(l[:kmax] - lc[:kmax]) / l[:kmax]
    metrics["error_eigenvalue"][0] = 0

    return metrics, l, lc


metrics, l, lc = coarsening_quality_(G, C, kmax=int(G.N/200))
np.nanmean(metrics['error_eigenvalue']) 

In [ ]:
l[:k], lc

In [ ]:
l

In [ ]:
poro_d = poro
perm_d = permx

poro_d_2 = coarsen_vector(poro_d, C)
perm_d_2 = coarsen_vector(perm_d, C)
depth_d_2 = coarsen_vector(depth, C)

In [ ]:
g_dict = get_coarsening(Gall, Call, size=5)
num_of_cells = np.array([len(g) for g in g_dict.values()])

v = np.zeros_like(poro_d_2)
v.fill(dx*dy*dz)
volume = num_of_cells * v

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

y = poro.reshape(-1,1)
x = permx.reshape(-1,1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)

model.fit(x_train, y_train)

y_pred = model.predict(x_train)
mse = mean_squared_error(y_pred, y_train)
print('Train MSE:', mse)

y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print('Test MSE:', mse)

plt.scatter(x, y, color='red', s=50)
plt.scatter(x, model.predict(x), s=50, marker='x')



for ind, num in enumerate(num_of_cells):
    if num > 1:
        new_poro = model.predict(perm_d_2[ind].reshape(-1,1))[0]
        poro_d_2[ind] = new_poro

In [ ]:
poro_stat, perm_stat = [], []
for ind, num in enumerate(num_of_cells):
    for _ in range(num):
        poro_stat.append(poro_d_2[ind])
        perm_stat.append(perm_d_2[ind])

In [ ]:
plt.hist(poro_d  , alpha=0.5, density=True, label='Initial', bins=25)
plt.hist(poro_stat, alpha=0.5, density=True, label='Coarsed', bins=25)
plt.legend(loc='upper right')
plt.show()


In [ ]:
plt.hist(perm_d , alpha=0.5, density=True, label='Initial', bins=25)
plt.hist(perm_stat, alpha=0.5, density=True, label='Coarsed', bins=25)
plt.legend(loc='upper right')
plt.show()


In [ ]:
Gall[0].plot_signal(poro_d)
Gall[1].plot_signal(poro_d_2)


In [ ]:
Gall[0].plot_signal(perm_d)
Gall[1].plot_signal(perm_d_2)